In [ ]:
import pandas as pd
import pdfplumber
from pathlib import Path
from IPython.display import display
from tqdm import tqdm
import locale

In [ ]:
# List of keywords
keywords = [
    'Abrechnung',
    'Bargeldaus',
    'Bargeldauszahlung',
    'Bargeldein',
    'Bargeldeinzahlung',
    'Entgeltabrechnung',
    'Entgelte',
    'Geldaus.',
    'Gut. Überw.',
    'Gutschr.',
    'GutschriftÜberweisung',
    'Kartenzahlung',
    'Kontostand',
    'Lastschrift',
    'Überweisung',
    'Überweisung Echtzeit',
    'Überweisung online'
]


def extract_transactions(pdf_path, keywords):
    transactions = []  # List to hold all rows
    columns = []  # List to store column names
    with pdfplumber.open(pdf_path) as pdf:  # Open the PDF
        for page_number, page in enumerate(pdf.pages, start=1):  # Iterate through pages
            tables = page.extract_tables()  # Extract all tables on the page
            
            for table_index, table in enumerate(tables):  # Iterate through each table
                if page_number == 1:  # If it's the first page
                    columns = table[0]  # Extract columns from the first table
                    table = table[1:]  # Skip the first row in the first table (header row)
                else:
                    table = table[1:]  # Skip the first row (column names) for subsequent tables

                # Process the second row (data) in the table
                if table:
                    combined_data = table[0]  # Second row of the table
                    # Step 1: Replace all \n with spaces except for ones directly before a keyword
                    try:
                        # Find the index of the "Erläuterung" column
                        description_index = columns.index("Erläuterung")
                    except ValueError:
                        # Raise an error if the column is not found
                        raise ValueError("Error: The column 'Erläuterung' was not found in the header.")
                    
                    # Step 2: Remove \n from Erlaueterung list except before keyword
                    desc = table[0][description_index]
                    processed_desc = "".join(
                        f"\n{line}" if any(line.startswith(keyword) for keyword in keywords) else 
                        f" {line}"
                        for line in desc.split("\n")
                    ).strip()
                    table[0][description_index] = processed_desc

                    # Step 3: Split each string in combined_data by '\n'
                    split_cells = [cell.split("\n") for cell in combined_data]
                    # Access the description list using the dynamically determined index
                    description_list = split_cells[description_index]

                    # Process all columns before "Erläuterung"
                    for col_idx in range(description_index):
                        column_data = split_cells[col_idx]  # Get the current column's data

                        # Check if 'Kontostand' is found in the first or last element of the description list
                        if "Kontostand" in description_list[0]:
                            column_data.insert(0, column_data[0])  # Add a copy of the first value
                        if "Kontostand" in description_list[-1]:
                            column_data.append(column_data[-1])  # Add a copy of the last value

                        # Update the split_cells with the modified column data
                        split_cells[col_idx] = column_data

                    # Step 4: Transpose the lists to group by rows
                    zipped_rows = zip(*split_cells)

                    # Step 5: Convert zipped rows into a list for final use
                    split_rows = list(zipped_rows)

                    transactions.extend(split_rows)  # Append all split rows to transactions

    return pd.DataFrame(transactions, columns=columns)  # Return both columns and data as a dataframe  

def process_pdfs_in_folder(folder_path):
    dataframes = []  # List to store DataFrames
    folder = Path(folder_path)  # Create a Path object for the folder
    pdf_files = list(folder.glob("*.pdf"))  # List all PDFs in the folder that match the pattern

    # Filter out only PDFs with "Konto" and "Auszug" in the filename
    pdf_files = [file for file in pdf_files if "Konto" in file.name and "Auszug" in file.name]

    # Display progress bar using tqdm, with no extra new lines
    with tqdm(total=len(pdf_files), desc="Processing PDFs", unit="file", ncols=100, leave=True) as pbar:
        for file_path in pdf_files:
            try:
                df1 = extract_transactions(file_path, keywords)  # Extract transactions
                dataframes.append(df1)  # Add the DataFrame to the list
            except Exception as e:
                print(f"Error processing file {file_path.name}: {e}")
            pbar.set_postfix({"file": file_path.name})  # Display current file being processed
            pbar.update(1)  # Update progress bar by one step

    return dataframes



# Example usage
folder_path = "D:/Sparkasse-Duisburg-Kontoauszuege"  # Replace with the path to your folder containing PDFs

all_dataframes = process_pdfs_in_folder(folder_path)
all_dataframes[9].to_csv(f'{folder_path}/test.csv', index=False)

Processing PDFs: 100%|█| 45/45 [00:19<00:00,  2.28file/s, file=Konto_1302319718-Auszug_2024_0010.PDF


In [3]:
# Set the locale to German (Germany) to handle the German date and currency format
locale.setlocale(locale.LC_TIME, 'de_DE')  # For date parsing
locale.setlocale(locale.LC_NUMERIC, 'de_DE')  # For numeric parsing

def process_dataframes(in_dataframes):
    processed_dfs = []

    # Iterate through the input DataFrames
    for df2 in in_dataframes:
        # Step 1: Remove "Wert" column if it exists
        if 'Wert' in df2.columns:
            df2 = df2.drop(columns=['Wert'])

        # Step 2: Convert 'Datum' column to datetime, ensuring no empty spaces and raise error on failure
        if 'Datum' in df2.columns:
            df2['Datum'] = df2['Datum'].astype(str)  # Ensure all values are strings
            
            # Remove any spaces in the 'Datum' string
            df2['Datum'] = df2['Datum'].str.replace(' ', '', regex=False)
            
            # Try to parse the 'Datum' column, and raise an error if parsing fails
            try:
                df2['Datum'] = pd.to_datetime(df2['Datum'], errors='raise', dayfirst=True)
            except Exception as e:
                raise ValueError(f"Error parsing 'Datum' values: {e}")

        # Step 3: Convert 'Betrag EUR' to numeric format, ensuring correct parsing, raise error on failure
        if 'Betrag EUR' in df2.columns:
            df2['Betrag EUR'] = df2['Betrag EUR'].astype(str)  # Convert all values to strings
            
            # Try to parse the 'Betrag EUR' column, and raise an error if parsing fails
            try:
                df2['Betrag EUR'] = df2['Betrag EUR'].apply(
                    lambda x: locale.atof(f"{x[-1]}{x[:-1]}" if x[-1] in '+-' else x)
                )
            except Exception as e:
                raise ValueError(f"Error parsing 'Betrag EUR' values: {e}")

        # Step 4: Append the processed DataFrame to the result list
        processed_dfs.append(df2)

    # Combine all processed DataFrames into one
    combined_df = pd.concat(processed_dfs, ignore_index=True)

    # Step 5: Sort combined DataFrame by 'Datum' (earliest first)
    combined_df = combined_df.sort_values(by='Datum', ascending=True)

    # Initialize 'Saldo' column to None
    combined_df['Saldo'] = None

    # Step 6: Initialize current_balance with the first row's Betrag EUR value
    current_balance = combined_df.iloc[0]['Betrag EUR']
    combined_df.at[0, 'Saldo'] = current_balance


    # Step 7: Iterate starting from the second row
    for index, row in combined_df.iloc[1:].iterrows():
        if 'Kontostand' in row['Erläuterung']:
            # For 'Kontostand' rows, copy the previous balance (no accumulation)
            combined_df.at[index, 'Saldo'] = current_balance
        else:
            # For transaction rows, first set Saldo to the current balance, then update it
            current_balance += row['Betrag EUR']  # Update the balance after the 'Saldo' is assigned
            combined_df.at[index, 'Saldo'] = current_balance

    # Round the 'Saldo' column to two decimal places
    combined_df['Saldo'] = combined_df['Saldo'].round(2)

    # Step 8: Save the combined DataFrame as a CSV file
    folder_path = "D:/Sparkasse-Duisburg-Kontoauszuege"  # Modify this path to your folder path
    combined_df.to_csv(f'{folder_path}/transactions.csv', index=False)

    return combined_df

# Example: Process all dataframes and generate the combined dataframe
all_transactions = process_dataframes(all_dataframes)


In [4]:
print(all_transactions.dtypes)


Datum          datetime64[ns]
Erläuterung            object
Betrag EUR            float64
Saldo                  object
dtype: object
